Задание 1.

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [63]:
KEYWORDS = ['python', 'парсинг']
URL = 'https://habr.com/ru/all/'
req = requests.get(URL)
soup = BeautifulSoup(req.text, 'html.parser')
posts = soup.find_all('article', class_='post')

In [ ]:
Так как вариант с preview-информацией разбирался на занятии, сразу делал необязательное

In [58]:
all_links = []
for post in posts:
    post_id = post.parent.attrs.get('id')
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    title_element = post.find('a', class_='post__title_link')
    link = title_element.attrs.get('href') # получаем ссылку для каждого поста
    soup_temp = BeautifulSoup(requests.get(link).text, 'html.parser')
    text_link = soup_temp.find('div', class_='post__text').text #получаем содержимое поста по ссылке
    for search_word in KEYWORDS: 
        if search_word in text_link: #проверяем наличие слова в тексте поста
            all_links.append(link)
            break
print(all_links)

['https://habr.com/ru/post/517056/', 'https://habr.com/ru/post/517044/']


In [60]:
habr_news = pd.DataFrame()
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()
    title = soup.find('span', class_='post__title-text').text
    text = soup.find('div', class_='post__text').text
    row = {'date': date, 'title': title, 'link': link, 'text': text}
    habr_news = pd.concat([habr_news, pd.DataFrame([row])]) 
habr_news   

,date,title,link,text
0,2020-08-29,Интерактивная визуализация алгоритмов на базе ...,https://habr.com/ru/post/517056/,Jupyter уже давно зарекомендовал себя как удоб...
0,2020-08-29,"Бот в телеграм, озвучивающий ваши эмоции в соо...",https://habr.com/ru/post/517044/,Привет!\nВ этой статье я опишу своего бота в т...


Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [62]:
URL = 'https://digibody.avast.com/v1/web/leaks'
# EMAIL = [xxx@x.ru, yyy@y.com]

req = requests.post(URL, json={"email": "caba7@mail.ru"})
req.text

'{"status":"ok","value":[{"leak_id":"1631f5e3-9b31-4a78-be91-111a01a87b56","username":"cabeh0k","domain":"","passwords":[{"password":null,"encrypted":false}],"leak_info":{"id":"1631f5e3-9b31-4a78-be91-111a01a87b56","date":1576195200000,"title":"Sensitive Source","description":"This source has been marked as sensitive due to one of the following reasons: Revealing the source may compromise an on-going investigation. The affected site is of a controversial nature but does not validate email addresses and could therefore be used to tarnish an employee\'s reputation.","number_of_entries":19009637,"source_references":[],"media_references":[],"domains":[""],"picture_url":null,"service_name":null},"marked_resolved":false,"leak_date":1576195200000,"passwords_count":1},{"leak_id":"2e6d3973-2110-4bdc-aca1-681d8b2eabfd","username":"caba7@mail.ru","domain":"storelp.ru","passwords":[{"password":null,"encrypted":true}],"leak_info":{"id":"2e6d3973-2110-4bdc-aca1-681d8b2eabfd","date":1529366400000,"ti